# Bleaching samples

## update sample type 
- everything is healthy or diseased right now, 
-> change to bleached based sampling notes
-> then crosscheck with colonydata?

In [1]:
import numpy as np
import pandas as pd
import os
import re

In [17]:
# upload sample and colony data 
sample=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv')
colony=pd.read_csv('~/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv')

In [19]:
sample.head()

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,Time_processed,Sample_type,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes
0,122022,BEL,CBC,12/5/22,CURLEW,4,NaN,77,OFAV,NaN,NaN,Core_frozen,18,Healthy,NaN,122022_BEL_CBC_T4_18_OFAV,NaN,NaN,NaN,NaN
1,92023,BEL,CBC,9/27/23,CBC30N,1,NaN,1,SSID,NaN,NaN,Core_RNAlater,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,NaN,NaN,NaN
2,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,2,PAST,NaN,NaN,Core_RNAlater,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
3,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,3,SSID,NaN,NaN,Core_RNAlater,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN
4,92023,BEL,CBC,9/25/23,CBC30N,1,NaN,12,PSTR,NaN,NaN,Core_RNAlater,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,NaN,NaN


In [20]:
colony.head()

,Unnamed: 0,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,...,082024_Percentage,Notes_082024,122024_Condition,122024_Percentage,Notes_122024,062025_Condition,062025_Percentage,Notes_062025,immune_y/n,checked_colonies
0,1.0,6/21/19,CBC30N,1,349,1,SSID,1.5,0.5,right,...,NaN,NaN,Not_visited,NaN,Diseased,Diseased,NaN,NaN,n,x
1,2.0,6/21/19,CBC30N,1,334,2,PAST,1.4,0.1,left,...,NaN,sample lower colony (original colony); inc old...,NaN,NaN,NaN,Healthy,NaN,NaN,y,x
2,3.0,6/21/19,CBC30N,1,346,3,SSID,6.3,0.8,right,...,NaN,some discoloration,DC,10%,30% mortality,Diseased,NaN,NaN,y,x
3,4.0,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.5,right,...,NaN,NaN,Dead,NaN,Dead,Dead,NaN,NaN,n,x
4,5.0,6/24/19,CBC30N,1,394,5,SSID,4.3,1.2,left,...,NaN,NaN,DC,10%,80% mortality,Diseased,NaN,NaN,n,x


In [23]:
# search for sampling notes with clp or clb 

#if col contains clb or clp 
# replace status with bleached tissue 
sample.loc[
    sample['Sampling_notes'].str.contains('CLP|CLB', case=False, na=False),'Health_status']= 'Bleached_Tissue'
# (code is in place replacement)

In [28]:
sample.loc[:,'Health_status']

0               Healthy
1       Diseased_Margin
2       Bleached_Tissue
3       Bleached_Tissue
4               Healthy
             ...       
2524            Healthy
2525            Healthy
2526            Healthy
2527            Healthy
2528            Healthy
Name: Health_status, Length: 2529, dtype: object

In [ ]:
# match sample type to colony statuses
# got from my cbc_metagenomics code

In [29]:
# filter for UML samples
sample_subset=sample[
    (sample['Sample_type'] == 'Core_EtOH') |
    (sample['Sample_type'] == 'Core_RNAlater')
]

In [30]:
# add colony ID - t# newtagnum species
sample_subset = sample_subset.copy()
sample_subset['TransectNum_str'] = 'T' + sample_subset['TransectNum'].astype(str)
sample_subset['colony_id'] = sample_subset[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
sample_subset.drop(columns='TransectNum_str', inplace=True)

In [31]:
# add colony ID - t# newtagnum species
colony = colony.copy()
colony['TransectNum_str'] = 'T' + colony['TransectNum'].astype(str)
colony['colony_id'] = colony[['TransectNum_str', 'NewTagNum', 'Species']].astype(str).agg('_'.join, axis=1)
colony.drop(columns='TransectNum_str', inplace=True)

In [115]:
# create function to crosscheck colony health with samples 

def status_match(group):
    # get the condition and list sample statuses of each colony at each monthyear (by row) 
    cond = group['colony_condition'].iloc[0]
    statuses = group['sample_condition'].dropna().tolist()

    # match colony conditions to sample conditions 
    # if colony_condition is healthy, sample = healthy 
    if cond == 'Healthy':
        return all(s == 'Healthy' for s in statuses)

    # if condition is diseased healthy = diseased_margin and diseased_tissue (but rn these are on 2 diff rows)
    elif cond == 'Diseased':
        return all(x in statuses for x in ['Diseased_Tissue', 'Diseased_Margin'])

    # if condition is clp | clb = bleached_tissue 
    elif cond in ['CLP','CLB']:
        return all(x in statuses for x in ['Bleached_Tissue'])
    
    # if condition is Not_visited, sample = NaN
    # if condition is Dead, sample = NaN 
    elif cond in ['Dead', 'Not_Visited','Not_visited','not_visited']:
        return all(pd.isna(s) for s in group['sample_condition'])

    # if condition is na, sample should also be na 
    elif pd.isna(cond):
        return all(pd.isna(s) for s in group['sample_condition'])

    # any mismatches return as false 
    else:
        return False

In [116]:
# create species list 
species_list=sample_subset['Species'].unique().tolist()
species_list

# ugh so many syntax errors...need to fix in 'sample_fixes'

['SSID',
 'PAST',
 'PSTR',
 'MCAV',
 'OANN',
 'DLAB',
 'OFAV',
 'CNAT',
 'DL',
 'MMEA',
 'PAST ',
 'Unknown',
 'OFAV ']

In [117]:
# extract condition columns 
cond_cols=['colony_id'] + colony.columns[colony.columns.str.contains('_Condition')].tolist()

In [118]:
# create for loop for each specie - using the fxn to match the colony and sample health statuses 
species_checks = {}

for specie in species_list: 
    # filter colony and sample data for each specie 
    filtered_colony=colony[colony['Species']==specie]

    # extract condition columns 
    small_colony= filtered_colony.loc[:,cond_cols]
    # and pivot
    conditions_long=small_colony.melt(id_vars='colony_id', 
                              var_name='Month_year', 
                              value_name='colony_condition')
    
    # extract month_year from column names to match sample data 
    conditions_long['Month_year'] = conditions_long['Month_year'].str.extract(r'(\d+)').astype(int)
   
    # repeat for sample data 
    filtered_samples=sample_subset[sample_subset['Species']==specie]
    filtered_samples=filtered_samples.loc[:,('colony_id','Month_year','Health_status','Sampling_notes','Tubelabel_species')] 
    # rename column for sample health statuses 
    filtered_samples['sample_condition']=filtered_samples['Health_status']
    
    # merge dfs by colony id and monthyear
    merged = pd.merge(
    conditions_long.merge(filtered_colony, on='colony_id', how='left'),
    filtered_samples,
    on=['colony_id', 'Month_year'],
    how='outer'
    )

    checks = []
        # check each 'group' (unique combos of colony and monthyear) and store results in 'checks' list
    for (colony_id, month), group in merged.groupby(['colony_id', 'Month_year']):
        checks.append({
            'colony_id': colony_id,
            'Date_InitialTag': group['Date_InitialTag'].iloc[0],
            'Month_year': month,
                
            # run function &
            # create col containing results of the function that matches statuses
            'Match': status_match(group),
                
            'colony_condition': group['colony_condition'].iloc[0],
            'sample_statuses': group['sample_condition'].tolist(),
            'sample_ids': group['Tubelabel_species'].tolist(),
            'mortality_date' : group['Date_DocumentedMortality'].iloc[0]
    })
    # convert to dataframe and store each species df in a dictionary 'species_checks'
    species_checks[specie]=pd.DataFrame(checks)
    

In [119]:
species_checks['MCAV']

,colony_id,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
0,T1_14_MCAV,6/24/19,12024,True,Not_Visited,[nan],[nan],12/2/22
1,T1_14_MCAV,6/24/19,22024,True,Not_Visited,[nan],[nan],12/2/22
2,T1_14_MCAV,6/24/19,42024,True,Not_Visited,[nan],[nan],12/2/22
3,T1_14_MCAV,6/24/19,52022,True,Diseased,"[Diseased_Margin, Diseased_Tissue]","[052022_BEL_CBC_T1_39_MCAV, 052022_BEL_CBC_T1_...",12/2/22
4,T1_14_MCAV,6/24/19,62019,True,Healthy,[Healthy],[062019_BEL_CBC_T1_17_MCAV],12/2/22
...,...,...,...,...,...,...,...,...
542,T6_9_MCAV,4/29/24,92023,True,NaN,[nan],[nan],Healthy
543,T6_9_MCAV,4/29/24,112023,True,NaN,[nan],[nan],Healthy
544,T6_9_MCAV,4/29/24,122022,True,NaN,[nan],[nan],Healthy
545,T6_9_MCAV,4/29/24,122023,True,NaN,[nan],[nan],Healthy


In [120]:
# check mismatches 
false_matches = {}
for specie in species_list:
    # make individual dfs per sp 
    df = pd.DataFrame(species_checks[specie])
    # filter for where the match is false 
    false = df[(df['Match'] == False) & (~df['colony_condition'].isna())]
    # exclude 122024 because no samples were taken 
    
    # store false matches in new dictionary with species name as key 
    false_matches[specie] = false

In [ ]:
# maybe add filter for immune vs not to exclude non-immune samples from immunity time points ?

In [121]:
false_matches['MCAV']

,colony_id,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
99,T1_7_MCAV,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_181_MCAV],12/1/24
114,T1_8_MCAV,6/24/19,122022,False,Diseased,[Diseased_Tissue],[122022_BEL_CBC_T1_144_MCAV],9/25/23
119,T2_53_MCAV,6/22/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1019_MCAV],Healthy
130,T2_55_MCAV,6/25/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T2_603_MCAV],Healthy
132,T2_55_MCAV,6/25/19,42024,False,Healthy,[Bleached_Tissue],[042024_BEL_CBC_T2_1016_MCAV],Healthy
141,T2_55_MCAV,6/25/19,122023,False,"CLP,CLB",[Bleached_Tissue],[122023_BEL_CBC_T2_518_MCAV],Healthy
142,T2_55_MCAV,6/25/19,122024,False,CLP,[nan],[nan],Healthy
159,T2_59_MCAV,6/21/19,52022,False,Diseased,[nan],[nan],Diseased
164,T2_59_MCAV,6/21/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T2_220_MCAV],Diseased
168,T2_59_MCAV,6/21/19,122024,False,CLP,[nan],[nan],Diseased


In [122]:
false_matches['PAST']

,colony_id,Date_InitialTag,Month_year,Match,colony_condition,sample_statuses,sample_ids,mortality_date
9,T1_13_PAST,6/24/19,112023,False,CLP,[nan],[nan],12/16/23
13,T1_19_PAST,6/21/19,12024,False,CLB,[nan],[nan],4/1/24
16,T1_19_PAST,6/21/19,52022,False,Diseased,[Diseased_Tissue],[052022_BEL_CBC_T1_52_PAST],4/1/24
21,T1_19_PAST,6/21/19,92023,False,CLB,[nan],[nan],4/1/24
26,T1_20_PAST,6/24/19,12024,False,CLP,[nan],[nan],4/26/24
39,T1_21_PAST,6/24/19,12024,False,CLP,[Healthy],[012024_BEL_CBC_T1_565_PAST],8/1/24
47,T1_21_PAST,6/24/19,92023,False,CLP,[Healthy],[092023_BEL_CBC_T1_172_PAST],8/1/24
50,T1_21_PAST,6/24/19,122023,False,"CLB,CLP",[Bleached_Tissue],[122023_BEL_CBC_T1_507_PAST],8/1/24
52,T1_2_PAST,6/21/19,12024,False,"CLP,CLB",[nan],[nan],Healthy
60,T1_2_PAST,6/21/19,92023,False,"CLP,CLB",[Bleached_Tissue],[092023_BEL_CBC_T1_171_PAST],Healthy


In [ ]:
false_matches['PAST']